In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import tensorflow as tf
# import shap
import plotly.express as px
import plotly.graph_objs as go
import os


In [ ]:
#testname   = "_prc_24_05_14Ext.csv"
path       = '/content/'
size_      = "500"
trigger_   = "trigger" + size_
# triggerCol = "mytrig_hi"
hi_lo      = "hi"
triggerCol = trigger_ + hi_lo
#modelname  = "model_"+triggerCol+"_1.keras"
#modelname  = "chkmodel_mytrig_"+hi_lo+"_1.keras"

#model_hi50  = "chkmodel_mytrig_hi83_a_9989_0031.keras"          #noTrain
#model_lo50  = "chkmodel_mytrig_lo83_a_9989_0031.keras"

#model_hi50  = "chkmodel_mytrig_hi83_b_9990_0027.keras"          #noTrain
#model_lo50  = "chkmodel_mytrig_lo83_b_9990_0033.keras"

#model_hi50  = "chkmodel_mytrig_hi91_dcba_9993_0025.keras"      #mitTrain
#model_lo50  = "chkmodel_mytrig_lo91_cbad_9991_0028.keras"

model_hi50  = "chkmodel_mytrig_hi67_best9_9992_0024.keras"     #mitTrain
model_lo50  = "chkmodel_mytrig_lo67_best9_9989_0026.keras"

model_hi20  = "chkmodel_mytrig_hi50_best8_9999_000797.keras"   #noTrain
model_lo20  = "chkmodel_mytrig_lo50_best1_9994_0021.keras"

#model_hi20  = "model_mytrig_hi20_1.keras"                      #noTrain
#model_lo20  = "model_mytrig_lo20_1.keras"


In [ ]:
# Funktion zum Erstellen von Datensätzen für LSTM
# Diese Funktion bereitet die Daten für das LSTM-Modell vor, indem sie Sequenzen (look_back) und die entsprechenden Zielwerte erstellt.
def create_dataset(dataset, look_back=1):
  # X wird die Liste der Eingabesequenzen für das LSTM sein.
  # Y wird die Liste der entsprechenden Zielwerte sein.
  X, Y = [], []
  # Iteriere durch das Dataset. Die Schleife endet früh genug, um sicherzustellen, dass für jede Sequenz
  # ein zukünftiger Zielwert (look_back Schritte entfernt) verfügbar ist.
  for i in range(len(dataset)-look_back-1):
    # Extrahiere eine Sequenz von 'look_back' Schritten aus dem Dataset.
    # Die letzten Spalte (der Zielwert) wird dabei ausgeschlossen.
    a = dataset[i:(i+look_back), :-1]
    X.append(a)
    # Extrahiere den Zielwert, der 'look_back' Schritte nach dem Ende der aktuellen Sequenz liegt.
    # Dies ist der Wert, den das LSTM vorhersagen soll.
    Y.append(dataset[i + look_back, -1])
  # Konvertiere die Listen X und Y in NumPy-Arrays.
  return np.array(X), np.array(Y)

In [ ]:
def create_day4lstm(path_, name_, triggercol_):
    df3  = pd.read_csv( path_ + name_ )
    mytrig = True if (triggercol_[:6] == "mytrig") else False
    print(triggercol_ + " mytrig? " + str(mytrig))
    if mytrig:
      dff  = pd.read_csv( path +          name_[:-7] +'Scorefix1.csv')
      df3c = pd.concat([df3, dff] , axis = 1)
    else:
      df3c = df3

    df3c["avg1000diff250"] =df3c["avg1000diffAct"] - df3c["avg250diffAct"]
    df3c["ago250diffAct"]  =df3c["ago250price"]   -df3c["priceMean"]
    df3c["ago1000diffAct"] =df3c["ago1000price"]  -df3c["priceMean"]
    df3c["trigger500hi"]   =df3c['trigger500'] .shift(+2, fill_value=0).clip(lower=0)
    df3c["trigger500lo"]   =df3c['trigger500'] .shift(+2, fill_value=0).clip(upper=0) * -1
    df3c["trigger1000hi"]  =df3c['trigger1000'].shift(+2, fill_value=0).clip(lower=0)
    df3c["trigger1000lo"]  =df3c['trigger1000'].shift(+2, fill_value=0).clip(upper=0) * -1
    if mytrig:
      df3c["mytrig_hi"]  =df3c['mytrig_'].shift(+2, fill_value=0).clip(lower=0)         # shift(+1
      df3c["mytrig_lo"]  =df3c['mytrig_'].shift(+2, fill_value=0).clip(upper=0) * -1

    #   scoresize = 25 : anzahl trades als kleinste auswertungsblock mit min, max, mean, etc...  (zum traden evtl. zu groß gewaehlt)
    #   avgsize   = 50 : anzahl trades die für avgXX werte genommenw werden
    #   alle price angaben sind relativ zum startprice des tages
    featr  = [  "timescore",      #  je kürzer der block desto wertvoller deshalb (3/timediff_sec)
                "volSum",         #  summe gehandelte trades im Block
                "prcdiff",        #  aenderung des Preises zum vorghrigen block (prc_last)
                "prcgrad",        #  Steigung der Preisaenderung (je steiler, desto wertvoller)
                "priceMean0",     #  avererage der letzten avgsize trades

                "ago250diffAct",
                "ago250diff",     #  aenderung des Preises vor 250 Bloecken zum vorherigen block (prc_last)
                "ago250grad",     #  steigung des Preises vor 250 Bloecken zum vorherigen block (prc_last)
                "avg250diff",
                "avg250grad",
                "avg250diffAct",  #  differenz des 250-Bloecke Avg.Preises zum actual Price
                "avg250crossAct", #  Schneiden der 250 line und der Act Line (-1 von unten nach oben, 1 von oben nqch unten)

                "ago1000diffAct",
                "ago1000diff",    #  dto.fuer 1000 Bloecke
                "ago1000grad",
                "avg1000diff",
                "avg1000grad",
                "avg1000diffAct",
                "avg1000crossAct",

                "avg1000diff250"]

    np_in = df3c[ featr ].to_numpy()[50:]
    np_trans = preprocessing.StandardScaler().fit_transform(np_in)
    data = np.column_stack((np_trans, df3c[ triggercol_ ].to_numpy()[50:]))

    train_size = int(len(data) * 0.95)
    train_data = data[:train_size]
    test_data  = data[train_size:]

    look_back = 10
    x_train, y_train = create_dataset(train_data, look_back)
    return x_train, y_train

In [ ]:
model_hi20 = tf.keras.models.load_model(path + model_hi20)
model_lo20 = tf.keras.models.load_model(path + model_lo20)
model_hi50 = tf.keras.models.load_model(path + model_hi50)
model_lo50 = tf.keras.models.load_model(path + model_lo50)

model_hi50.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10, 32)         │         6,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,413 (177.40 KB)

 Trainable params: 15,137 (59.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 30,276 (118.27 KB)

In [ ]:
# #########################################################################
# #########################################################################
usemytrig    = True  #False  #True
fixtrigger   = "mytrig_" if usemytrig else trigger_
traintrigger = "mytrig_" if usemytrig else trigger_ #Col # trigger_  >> bei mytrigger beide richtungen

testdatum  = "24_11_24"
testname2  = "_prc_"+testdatum+"Ext.csv"
X_test, Y_test = create_day4lstm(path, testname2, traintrigger) #triggerCol)

predict_lo50 = model_lo50.predict(X_test)

predict_hi20 = model_hi20.predict(X_test)
predict_lo20 = model_lo20.predict(X_test)
predict_hi50 = model_hi50.predict(X_test)
#predict_lo50 = model_lo50.predict(X_test)

tt_hi20 = predict_hi20.reshape(-1)
tt_lo20 = predict_lo20.reshape(-1)
tt_hi50 = predict_hi50.reshape(-1)
tt_lo50 = predict_lo50.reshape(-1)

tt_hi20_idx = np.where(tt_hi20 > 0.5)[0]
tt_lo20_idx = np.where(tt_lo20 > 0.5)[0]
tt_hi50_idx = np.where(tt_hi50 > 0.5)[0]
tt_lo50_idx = np.where(tt_lo50 > 0.5)[0]

mytrig_ mytrig? True
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [ ]:
# ############################################################
# prepare daraframe for grafik output

df4c = pd.read_csv(path + testname2)
df4c["trigger250hi"]  = df4c['trigger250'] .shift(+2, fill_value=0).clip(lower=0)
df4c["trigger250lo"]  = df4c['trigger250'] .shift(+2, fill_value=0).clip(upper=0) * -1
df4c["trigger500hi"]  = df4c['trigger500'] .shift(+2, fill_value=0).clip(lower=0)
df4c["trigger500lo"]  = df4c['trigger500'] .shift(+2, fill_value=0).clip(upper=0) * -1
df4c["trigger1000hi"] = df4c['trigger1000'].shift(+2, fill_value=0).clip(lower=0)
df4c["trigger1000lo"] = df4c['trigger1000'].shift(+2, fill_value=0).clip(upper=0) * -1

if usemytrig:
  dff   = pd.read_csv( path +          testname2[:-7] +'Scorefix1.csv')
  df4c2 = pd.concat([df4c, dff] , axis = 1)
else:
  df4c2 = df4c

In [ ]:
# show grafik with plotly express

df4d = df4c2[50+10-2:].reset_index(drop=True)

fig4c = px.line(df4d,  title=testname2, y=["priceMin0", "priceMax0" ]) #ago250wavg0","ago1000wavg0" ]) #,"pricein250n" ])   #,"dif1000N" ])  #"max1000N","min1000N"])
fig4c.add_trace(go.Scatter(y = df4d['max'+size_+'_'],  line_color = 'gray', name="upper fwd" )) #, line = {'dash': 'dash'} ))
fig4c.add_trace(go.Scatter(y = df4d['min'+size_+'_'],  line_color = 'gray', name="lower fwd"  )) #, line = {'dash': 'dash'} ))  #   opacity = 0.5))
fig4c.add_trace(go.Bar    (y = df4d[fixtrigger]*100, width=2, name = "trigger"))
fig4c.add_trace(go.Bar    (y = tt_hi50* 1000 ,  width=1, name = "high(67)" ))
fig4c.add_trace(go.Bar    (y = tt_lo50*-1000 ,  width=1, name = "low(67)"))
fig4c.show()

fig4d = px.line(df4d,  title=testname2, y=["priceMin0", "priceMax0" ]) #ago250wavg0","ago1000wavg0" ]) #,"pricein250n" ])   #,"dif1000N" ])  #"max1000N","min1000N"])
fig4d.add_trace(go.Scatter(y = df4d['max'+size_+'_'],  line_color = 'gray', name="upper fwd" )) #, line = {'dash': 'dash'} ))
fig4d.add_trace(go.Scatter(y = df4d['min'+size_+'_'],  line_color = 'gray', name="lower fwd"  )) #, line = {'dash': 'dash'} ))  #   opacity = 0.5))
fig4d.add_trace(go.Bar    (y = df4d[fixtrigger]*100, width=2, name = "trigger"))
fig4d.add_trace(go.Bar    (y = tt_hi20* 1000 ,  width=1, name = "high(50)" ))
fig4d.add_trace(go.Bar    (y = tt_lo20*-1000 ,  width=1, name = "low(50)"))
fig4d.show()

In [ ]:
# ############################################################
# ############################################################
# ############################################################
# prepare for output of metrics and performance

triggerZielHi = np.where(Y_test >0.5)[0] + 2  #>> finale ziel um 2 verschoben
triggerZielLo = np.where(Y_test<-0.5)[0] + 2

triggerZielLo

array([ 59,  87, 171, 218, 245])

In [ ]:
def print_metriken(trigger_my, trigger_ziel, header, wfile):# calc metriken
  treffer_count = 0
  for number in trigger_my:
      found_in_range = False           # Setzen einer Markierung (Flag), um zu prüfen, ob die Zahl in einem Bereich gefunden wurde
      for zielindex in trigger_ziel:    # Schleife durch jeden Bereich (start, end) im Array A
          if (zielindex-2) <= number <= (zielindex+2):   # Prüfen, ob die Zahl innerhalb des aktuellen Bereichs liegt (inklusive der Grenzen)
              found_in_range = True    # Wenn die Zahl gefunden wurde, wird die innere Schleife verlassen, um mit der nächsten Zahl in B weiterzumachen.
              break
      if found_in_range:               # Wenn die Markierung 'found_in_range' auf True gesetzt wurde, wird der Zähler erhöht
          treffer_count += 1

  notreffer_count     = len(trigger_my)  - treffer_count
  zielnotfound_count  = len(trigger_ziel)- treffer_count

  praezision = 0 if (len(trigger_my)<=0) else treffer_count / len(trigger_my)                           #Praezision = True Positives​ / (True Positives + False Positives)
  recall     = treffer_count / (treffer_count + zielnotfound_count)      #Recall = True Positives​ / (True Positives + False Negatives)
  f1_score   = 0 if (praezision + recall <= 0) else (2 * praezision * recall) / (praezision + recall)        #F1−Score = 2 x  (Praezision + Recall) / (Praezision x Recall​)

  print("##### " + header)
  print(str(len(trigger_my))    + " triggerMy_count")
  print(str(len(trigger_ziel))  + " triggerZiel_count")
  print(str(treffer_count)      + " trigger_treffer_count")
  print(str(notreffer_count)    + " trigger_no_vorgabe_count")
  print(str(zielnotfound_count) + " vorgabe_notfound_count")
  print("Praezision : " + str(praezision) + " / recall : " + str(recall) + " / f1_score : " + str(f1_score) )
  wfile.write( f",{header},{len(trigger_my)},{len(trigger_ziel)},{treffer_count}" )


In [ ]:
  with open(path + 'out_metrics.txt', 'a') as wfile:
    wfile.write(testdatum);
    print(testname2+" - "+testdatum)
    print_metriken(tt_hi50_idx, triggerZielHi, "HI 67",wfile)
    print_metriken(tt_lo50_idx, triggerZielLo, "LO 67",wfile)
    print_metriken(tt_hi20_idx, triggerZielHi, "HI 50",wfile)
    print_metriken(tt_lo20_idx, triggerZielLo, "Lo 50",wfile)
    wfile.write("\n");

_prc_24_11_27Ext.csv - 24_11_27
##### HI 67
5 triggerMy_count
6 triggerZiel_count
2 trigger_treffer_count
3 trigger_no_vorgabe_count
4 vorgabe_notfound_count
Praezision : 0.4 / recall : 0.3333333333333333 / f1_score : 0.3636363636363636
##### LO 67
0 triggerMy_count
5 triggerZiel_count
0 trigger_treffer_count
0 trigger_no_vorgabe_count
5 vorgabe_notfound_count
Praezision : 0 / recall : 0.0 / f1_score : 0
##### HI 50
3 triggerMy_count
6 triggerZiel_count
1 trigger_treffer_count
2 trigger_no_vorgabe_count
5 vorgabe_notfound_count
Praezision : 0.3333333333333333 / recall : 0.16666666666666666 / f1_score : 0.2222222222222222
##### Lo 50
5 triggerMy_count
5 triggerZiel_count
1 trigger_treffer_count
4 trigger_no_vorgabe_count
4 vorgabe_notfound_count
Praezision : 0.2 / recall : 0.2 / f1_score : 0.20000000000000004


In [ ]:
def print_performance(trigger_my, header, up_down_fkt, wfile):# calc metriken
  print("##### " + header)
  midprices = df4d["priceMean0"].to_numpy()
  treffer_count = 0
  sumdiff = maxdiff = mindiff = 0
  for idx in trigger_my:
      midprc0 = midprices[idx]
      midprc9 = midprices[idx+10]
      prcdiff = (midprc9 - midprc0) * up_down_fkt
      sumdiff = sumdiff + prcdiff
      maxdiff = maxdiff if (prcdiff < maxdiff) else prcdiff
      mindiff = mindiff if (prcdiff > mindiff) else prcdiff

      print (str(midprc0) + " - " + str(midprc9) + " diff: " + str( prcdiff ))

      if prcdiff > 50:            # Wenn die Markierung 'found_in_range' auf True gesetzt wurde, wird der Zähler erhöht
          treffer_count += 1

  praezision_my = 0 if (len(trigger_my)<=0) else treffer_count / len(trigger_my)
  print (str(treffer_count) + " von " + str(len(trigger_my)) + " : in 10 bloecken +5 punkte")
  print(str(praezision_my) + " : praezision_beiPerformance ")
  print(str(sumdiff/10) + " sumdiff in fdax punkten")
  wfile.write( f",{header},{len(trigger_my)},{treffer_count},{sumdiff},{maxdiff},{mindiff}" )


In [ ]:
with open(path + 'out_stratbasis.txt', 'a') as wfile:
  wfile.write(testdatum);
  print(testname2)
  print_performance(tt_hi50_idx, "HI 67",  1, wfile)
  print_performance(tt_lo50_idx, "LO 67", -1, wfile)
  print_performance(tt_hi20_idx, "HI 50",  1, wfile)
  print_performance(tt_lo20_idx, "LO 50", -1, wfile)
  wfile.write("\n");

_prc_24_11_27Ext.csv
##### HI 67
-672 - -536 diff: 136
-597 - -355 diff: 242
-665 - -785 diff: -120
-302 - -112 diff: 190
-388 - 22 diff: 410
4 von 5 : in 10 bloecken +5 punkte
0.8 : praezision_beiPerformance 
85.8 sumdiff in fdax punkten
##### LO 67
0 von 0 : in 10 bloecken +5 punkte
0 : praezision_beiPerformance 
0.0 sumdiff in fdax punkten
##### HI 50
-672 - -536 diff: 136
-561 - -403 diff: 158
-67 - -332 diff: -265
2 von 3 : in 10 bloecken +5 punkte
0.6666666666666666 : praezision_beiPerformance 
2.9 sumdiff in fdax punkten
##### LO 50
-165 - -307 diff: 142
-37 - 45 diff: -82
112 - 260 diff: -148
183 - 172 diff: 11
0 - 208 diff: -208
1 von 5 : in 10 bloecken +5 punkte
0.2 : praezision_beiPerformance 
-28.5 sumdiff in fdax punkten


In [ ]:
def print_performance_limit(trigger_my, header, up_down_fkt, wfile):# calc metriken
  print("##### " + header)
  midprices = df4d["priceMean0"].to_numpy()
  treffer_count = 0
  sumdiff = maxdiff = mindiff = 0
  limit_tics = 150
  limit_akt  =  0
  for idx in trigger_my:
      midprc0     = midprices[idx]
      limit_start = midprc0 - (limit_tics * up_down_fkt)   # start limit
      limit_akt   = limit_start
      print( ">>> stratidx " + str(idx) + " prc " + str(midprc0) + " limit " + str(limit_akt) )

      notstopped = True
      fwdidx    = idx + 1
      midprc_akt=0
      while notstopped:
          midprc_akt  = midprices[fwdidx]
          if  fwdidx+1 >= len(midprices) or (midprc_akt - limit_akt) * up_down_fkt < 0:   # limit erreicht?
              midprc9 = limit_akt
              notstopped = False

          if limit_tics > 50 : limit_tics -= 5
          limit_new   = midprc_akt - (limit_tics * up_down_fkt)
          if  (limit_new - limit_akt) * up_down_fkt > 0 and (limit_new - limit_start) * up_down_fkt > 0:
              limit_akt = limit_new

          #print( "idx " + str(fwdidx) + " prc " + str(midprc_akt) + " limit " + str(limit_akt) )
          fwdidx += 1

      print( "idx " + str(fwdidx) + " prc " + str(midprc_akt) + " limit " + str(limit_akt) )
      prcdiff = (midprc9 - midprc0) * up_down_fkt
      sumdiff = sumdiff + prcdiff
      maxdiff = maxdiff if (prcdiff < maxdiff) else prcdiff
      mindiff = mindiff if (prcdiff > mindiff) else prcdiff

      print (str(midprc0) + " - " + str(midprc9) + " diff: " + str( prcdiff ))

      if prcdiff > 50:            # Wenn die Markierung 'found_in_range' auf True gesetzt wurde, wird der Zähler erhöht
          treffer_count += 1
  print("=======================")
  print (str(treffer_count) + " von " + str(len(trigger_my)) + " : im trade +5 punkte")
  print(str(sumdiff/10) + " sumdiff in fdax punkten")
  wfile.write( f",{header},{len(trigger_my)},{treffer_count},{sumdiff},{maxdiff},{mindiff}" )
  print(" ")

In [ ]:
with open(path + 'out_stratlimit.txt', 'a') as wfile:
  wfile.write(testdatum);
  print(testname2)
  print_performance_limit(tt_hi50_idx, "HI 67",  1, wfile)
  print_performance_limit(tt_lo50_idx, "LO 67", -1, wfile)
  print_performance_limit(tt_hi20_idx, "HI 50",  1, wfile)
  print_performance_limit(tt_lo20_idx, "LO 50", -1, wfile)
  wfile.write("\n");

_prc_24_11_27Ext.csv
##### HI 67
>>> stratidx 13 prc -672 limit -822
idx 48 prc -239 limit -237
-672 - -237 diff: 435
>>> stratidx 78 prc -597 limit -647
idx 92 prc -480 limit -401
-597 - -401 diff: 196
>>> stratidx 93 prc -665 limit -715
idx 99 prc -745 limit -715
-665 - -715 diff: -50
>>> stratidx 131 prc -302 limit -352
idx 139 prc -115 limit -112
-302 - -112 diff: 190
>>> stratidx 190 prc -388 limit -438
idx 213 prc 157 limit 182
-388 - 182 diff: 570
4 von 5 : im trade +5 punkte
134.1 sumdiff in fdax punkten
 
##### LO 67
0 von 0 : im trade +5 punkte
0.0 sumdiff in fdax punkten
 
##### HI 50
>>> stratidx 13 prc -672 limit -822
idx 48 prc -239 limit -237
-672 - -237 diff: 435
>>> stratidx 77 prc -561 limit -611
idx 80 prc -620 limit -611
-561 - -611 diff: -50
>>> stratidx 182 prc -67 limit -117
idx 187 prc -125 limit -99
-67 - -99 diff: -32
1 von 3 : im trade +5 punkte
35.3 sumdiff in fdax punkten
 
##### LO 50
>>> stratidx 59 prc -165 limit -15
idx 82 prc -565 limit -570
-165 - -57

In [ ]:
# ###########################################
# ###########################################
# print output of small plots with plotly

fig5 = px.line(df4d, y= ["priceMean0"], height=300)
fig5.add_trace(go.Bar(y = tt_hi50 *  1000, width=1, name = "high(67)"))
fig5.add_trace(go.Bar(y = tt_lo50 * -1000, width=1, name = "low(67)"))
fig5.add_trace(go.Bar(y = df4d[fixtrigger]*100, width=2, name= "trigger" ))
fig5.show()

fig6 = px.line(df4d, y= ["priceMean0"], height=300)
fig6.add_trace(go.Bar(y = tt_hi20 *  1000, width=1, name="high(50)"))
fig6.add_trace(go.Bar(y = tt_lo20 * -1000, width=1, name="low(50)"))
fig6.add_trace(go.Bar(y = df4d[fixtrigger]*100, width=2, name= "trigger" ))
fig6.show()